# Advanced Analytics and Forecasting

This notebook covers advanced statistical analysis and forecasting techniques for LLM cost prediction.

## Learning Objectives
- Perform statistical analysis on cost data
- Detect trends and seasonality
- Forecast future costs using Prophet
- Identify anomalies and outliers

## Prerequisites
```bash
pip install pandas matplotlib prophet scipy scikit-learn statsmodels
```

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Prophet for time series forecasting
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    print("Prophet not available. Install with: pip install prophet")
    PROPHET_AVAILABLE = False

# Statsmodels for statistical analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load and Prepare Data

In [ ]:
# API Configuration
BASE_URL = 'http://localhost:3000/api'
API_KEY = 'your-api-key-here'
HEADERS = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# Fetch 90 days of data for better analysis
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

params = {
    'start_date': start_date.isoformat(),
    'end_date': end_date.isoformat(),
    'limit': 5000
}

response = requests.get(f'{BASE_URL}/cost-tracking', headers=HEADERS, params=params)
cost_data = response.json()

df = pd.DataFrame(cost_data['data'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date
df = df.sort_values('timestamp')

print(f"Loaded {len(df)} records from {start_date.date()} to {end_date.date()}")
df.head()

In [ ]:
# Prepare daily aggregated data
daily_df = df.groupby('date').agg({
    'total_cost': 'sum',
    'input_tokens': 'sum',
    'output_tokens': 'sum',
    'request_id': 'count'
}).reset_index()

daily_df.columns = ['date', 'total_cost', 'input_tokens', 'output_tokens', 'request_count']
daily_df['total_tokens'] = daily_df['input_tokens'] + daily_df['output_tokens']
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df = daily_df.set_index('date').sort_index()

print(f"Daily aggregated data: {len(daily_df)} days")
daily_df.head()

## 2. Statistical Analysis

In [ ]:
# Descriptive statistics
print("Daily Cost Statistics:")
print("="*50)
print(f"Mean: ${daily_df['total_cost'].mean():.2f}")
print(f"Median: ${daily_df['total_cost'].median():.2f}")
print(f"Std Dev: ${daily_df['total_cost'].std():.2f}")
print(f"Min: ${daily_df['total_cost'].min():.2f}")
print(f"Max: ${daily_df['total_cost'].max():.2f}")
print(f"\nSkewness: {daily_df['total_cost'].skew():.2f}")
print(f"Kurtosis: {daily_df['total_cost'].kurtosis():.2f}")

# Coefficient of variation
cv = (daily_df['total_cost'].std() / daily_df['total_cost'].mean()) * 100
print(f"\nCoefficient of Variation: {cv:.2f}%")

In [ ]:
# Test for normality
statistic, p_value = stats.normaltest(daily_df['total_cost'])
print(f"Normality Test (D'Agostino-Pearson):")
print(f"Test Statistic: {statistic:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Result: {'Normally distributed' if p_value > 0.05 else 'Not normally distributed'} (α=0.05)")

# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with normal distribution overlay
axes[0].hist(daily_df['total_cost'], bins=30, density=True, alpha=0.7, edgecolor='black')
mu, sigma = daily_df['total_cost'].mean(), daily_df['total_cost'].std()
x = np.linspace(daily_df['total_cost'].min(), daily_df['total_cost'].max(), 100)
axes[0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Distribution')
axes[0].set_title('Daily Cost Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Daily Cost ($)')
axes[0].set_ylabel('Density')
axes[0].legend()

# Q-Q plot
stats.probplot(daily_df['total_cost'], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Trend Detection and Decomposition

In [ ]:
# Augmented Dickey-Fuller test for stationarity
result = adfuller(daily_df['total_cost'].dropna())
print("Augmented Dickey-Fuller Test:")
print(f"ADF Statistic: {result[0]:.4f}")
print(f"P-value: {result[1]:.4f}")
print(f"Critical Values:")
for key, value in result[4].items():
    print(f"  {key}: {value:.3f}")
print(f"\nResult: {'Stationary' if result[1] < 0.05 else 'Non-stationary'} (α=0.05)")

In [ ]:
# Time series decomposition (if we have enough data)
if len(daily_df) >= 14:
    decomposition = seasonal_decompose(daily_df['total_cost'], model='additive', period=7)
    
    fig, axes = plt.subplots(4, 1, figsize=(14, 10))
    
    decomposition.observed.plot(ax=axes[0], title='Observed', color='blue')
    axes[0].set_ylabel('Cost ($)')
    
    decomposition.trend.plot(ax=axes[1], title='Trend', color='red')
    axes[1].set_ylabel('Cost ($)')
    
    decomposition.seasonal.plot(ax=axes[2], title='Seasonal (Weekly)', color='green')
    axes[2].set_ylabel('Cost ($)')
    
    decomposition.resid.plot(ax=axes[3], title='Residual', color='orange')
    axes[3].set_ylabel('Cost ($)')
    axes[3].set_xlabel('Date')
    
    for ax in axes:
        ax.grid(True, alpha=0.3)
    
    plt.suptitle('Time Series Decomposition', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient data for seasonal decomposition (need at least 14 days)")

## 4. Rolling Statistics and Moving Averages

In [ ]:
# Calculate rolling statistics
daily_df['ma_7'] = daily_df['total_cost'].rolling(window=7).mean()
daily_df['ma_14'] = daily_df['total_cost'].rolling(window=14).mean()
daily_df['rolling_std_7'] = daily_df['total_cost'].rolling(window=7).std()

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(daily_df.index, daily_df['total_cost'], label='Daily Cost', alpha=0.5, marker='o', markersize=3)
ax.plot(daily_df.index, daily_df['ma_7'], label='7-Day MA', linewidth=2)
ax.plot(daily_df.index, daily_df['ma_14'], label='14-Day MA', linewidth=2)

ax.set_title('Cost Trends with Moving Averages', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cost ($)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Volatility analysis
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(daily_df.index, daily_df['rolling_std_7'], label='7-Day Rolling Std Dev', 
        linewidth=2, color='coral')
ax.fill_between(daily_df.index, daily_df['rolling_std_7'], alpha=0.3, color='coral')

ax.set_title('Cost Volatility (7-Day Rolling Standard Deviation)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Standard Deviation ($)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average Daily Volatility: ${daily_df['rolling_std_7'].mean():.2f}")
print(f"Maximum Volatility: ${daily_df['rolling_std_7'].max():.2f}")

## 5. Anomaly Detection

In [ ]:
# Statistical anomaly detection using Z-score
daily_df['z_score'] = np.abs(stats.zscore(daily_df['total_cost']))
daily_df['is_outlier_zscore'] = daily_df['z_score'] > 3

# IQR method
Q1 = daily_df['total_cost'].quantile(0.25)
Q3 = daily_df['total_cost'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
daily_df['is_outlier_iqr'] = (daily_df['total_cost'] < lower_bound) | (daily_df['total_cost'] > upper_bound)

# Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
daily_df['is_outlier_iforest'] = iso_forest.fit_predict(daily_df[['total_cost']].values) == -1

print("Anomaly Detection Results:")
print(f"Z-Score method (|z| > 3): {daily_df['is_outlier_zscore'].sum()} anomalies")
print(f"IQR method: {daily_df['is_outlier_iqr'].sum()} anomalies")
print(f"Isolation Forest: {daily_df['is_outlier_iforest'].sum()} anomalies")

In [ ]:
# Visualize anomalies
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

methods = [
    ('is_outlier_zscore', 'Z-Score Method (|z| > 3)'),
    ('is_outlier_iqr', 'IQR Method'),
    ('is_outlier_iforest', 'Isolation Forest')
]

for idx, (col, title) in enumerate(methods):
    ax = axes[idx]
    
    # Plot normal points
    normal = daily_df[~daily_df[col]]
    ax.scatter(normal.index, normal['total_cost'], alpha=0.6, s=50, label='Normal')
    
    # Plot anomalies
    anomalies = daily_df[daily_df[col]]
    if len(anomalies) > 0:
        ax.scatter(anomalies.index, anomalies['total_cost'], 
                  color='red', s=100, marker='X', label='Anomaly', zorder=5)
    
    ax.set_title(f'Anomaly Detection: {title}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Daily Cost ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)

axes[2].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Detailed anomaly report
anomalies = daily_df[daily_df['is_outlier_zscore'] | daily_df['is_outlier_iqr'] | daily_df['is_outlier_iforest']]

if len(anomalies) > 0:
    print("\nDetailed Anomaly Report:")
    print("="*80)
    for date, row in anomalies.iterrows():
        print(f"\nDate: {date.date()}")
        print(f"  Cost: ${row['total_cost']:.2f}")
        print(f"  Z-Score: {row['z_score']:.2f}")
        print(f"  Requests: {row['request_count']:.0f}")
        print(f"  Total Tokens: {row['total_tokens']:.0f}")
        print(f"  Detection Methods: ", end="")
        methods_detected = []
        if row['is_outlier_zscore']: methods_detected.append('Z-Score')
        if row['is_outlier_iqr']: methods_detected.append('IQR')
        if row['is_outlier_iforest']: methods_detected.append('Isolation Forest')
        print(', '.join(methods_detected))
else:
    print("No anomalies detected in the dataset.")

## 6. Cost Forecasting with Prophet

In [ ]:
if PROPHET_AVAILABLE:
    # Prepare data for Prophet
    prophet_df = daily_df.reset_index()[['date', 'total_cost']]
    prophet_df.columns = ['ds', 'y']
    
    # Create and fit model
    model = Prophet(
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=False if len(prophet_df) < 365 else True,
        changepoint_prior_scale=0.05
    )
    
    model.fit(prophet_df)
    print("Prophet model trained successfully!")
    
    # Make future predictions (30 days)
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)
    
    print(f"\nForecast created for next 30 days")
    print(f"Predicted total cost for next 30 days: ${forecast.tail(30)['yhat'].sum():.2f}")
else:
    print("Prophet not available. Skipping forecasting.")

In [ ]:
if PROPHET_AVAILABLE:
    # Plot forecast
    fig = model.plot(forecast, figsize=(14, 6))
    ax = fig.gca()
    ax.set_title('Cost Forecast (30 Days)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Daily Cost ($)', fontsize=12)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Plot components
    fig = model.plot_components(forecast, figsize=(14, 8))
    plt.tight_layout()
    plt.show()

In [ ]:
if PROPHET_AVAILABLE:
    # Forecast summary
    future_forecast = forecast.tail(30)
    
    print("\n30-Day Forecast Summary:")
    print("="*60)
    print(f"Expected Daily Average: ${future_forecast['yhat'].mean():.2f}")
    print(f"Expected Total Cost: ${future_forecast['yhat'].sum():.2f}")
    print(f"\nConfidence Interval (95%):")
    print(f"  Lower Bound (Total): ${future_forecast['yhat_lower'].sum():.2f}")
    print(f"  Upper Bound (Total): ${future_forecast['yhat_upper'].sum():.2f}")
    print(f"\nHighest Expected Day: ${future_forecast['yhat'].max():.2f} on {future_forecast.loc[future_forecast['yhat'].idxmax(), 'ds'].date()}")
    print(f"Lowest Expected Day: ${future_forecast['yhat'].min():.2f} on {future_forecast.loc[future_forecast['yhat'].idxmin(), 'ds'].date()}")

## 7. Correlation Analysis

In [ ]:
# Calculate correlations
correlation_cols = ['total_cost', 'request_count', 'total_tokens', 'input_tokens', 'output_tokens']
correlation_matrix = daily_df[correlation_cols].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nKey Correlations with Total Cost:")
print(correlation_matrix['total_cost'].sort_values(ascending=False))

## 8. Growth Rate Analysis

In [ ]:
# Calculate week-over-week growth
daily_df['cost_pct_change'] = daily_df['total_cost'].pct_change() * 100

# Weekly aggregation
weekly_df = daily_df.resample('W').agg({
    'total_cost': 'sum',
    'request_count': 'sum',
    'total_tokens': 'sum'
})

weekly_df['wow_growth'] = weekly_df['total_cost'].pct_change() * 100

print("Week-over-Week Growth Rates:")
print("="*60)
for idx, row in weekly_df.iterrows():
    if pd.notna(row['wow_growth']):
        print(f"Week ending {idx.date()}: {row['wow_growth']:+.1f}% (${row['total_cost']:.2f})")

avg_growth = weekly_df['wow_growth'].mean()
print(f"\nAverage Weekly Growth Rate: {avg_growth:+.1f}%")

In [ ]:
# Visualize growth
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Weekly costs with growth
ax1 = axes[0]
ax1.bar(weekly_df.index, weekly_df['total_cost'], alpha=0.7, width=5)
ax1.set_title('Weekly Total Costs', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cost ($)')
ax1.grid(True, alpha=0.3, axis='y')

# Growth rate
ax2 = axes[1]
colors = ['green' if x >= 0 else 'red' for x in weekly_df['wow_growth']]
ax2.bar(weekly_df.index, weekly_df['wow_growth'], alpha=0.7, width=5, color=colors)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Week-over-Week Growth Rate', fontsize=12, fontweight='bold')
ax2.set_ylabel('Growth Rate (%)')
ax2.set_xlabel('Week Ending')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. Export Analysis Results

In [ ]:
# Export results
output_dir = 'advanced_analytics_output'
import os
os.makedirs(output_dir, exist_ok=True)

# Export anomalies
if len(anomalies) > 0:
    anomalies.to_csv(f'{output_dir}/anomalies.csv')
    print(f"Saved: {output_dir}/anomalies.csv")

# Export forecast
if PROPHET_AVAILABLE:
    future_forecast.to_csv(f'{output_dir}/forecast_30days.csv', index=False)
    print(f"Saved: {output_dir}/forecast_30days.csv")

# Export weekly summary
weekly_df.to_csv(f'{output_dir}/weekly_summary.csv')
print(f"Saved: {output_dir}/weekly_summary.csv")

# Export correlation matrix
correlation_matrix.to_csv(f'{output_dir}/correlation_matrix.csv')
print(f"Saved: {output_dir}/correlation_matrix.csv")

print(f"\nAll results exported to '{output_dir}/' directory")

## Summary

In this notebook, you learned:
- Statistical analysis techniques for cost data
- Trend detection and time series decomposition
- Multiple anomaly detection methods
- Cost forecasting using Prophet
- Correlation and growth rate analysis

## Next Steps
- **03_cost_optimization.ipynb** - Apply insights for cost optimization
- **04_custom_reports.ipynb** - Build custom reporting dashboards
- **05_ml_forecasting.ipynb** - Advanced ML forecasting techniques